In [1]:
import pandas as pd
df = pd.read_csv("/content/creditcard.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/content/creditcard.csv'

In [ ]:
pip install numpy pandas matplotlib seaborn scikit-learn imbalanced-learn xgboost joblib


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns; from sklearn.model_selection import train_test_split, GridSearchCV; from sklearn.preprocessing import StandardScaler; from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score; from sklearn.ensemble import RandomForestClassifier; from xgboost import XGBClassifier; from imblearn.over_sampling import SMOTE; from imblearn.under_sampling import RandomUnderSampler; from imblearn.pipeline import Pipeline as ImbPipeline; import joblib; RANDOM_STATE=42; sns.set(style="whitegrid")


 Load data

In [ ]:

df = pd.read_csv("creditcard.csv")
X, y = df.drop(['Class','Time'], axis=1), df['Class']
features = X.columns.tolist()


train/test split

In [ ]:
df = df.dropna(subset=['Class'])
X, y = df.drop(['Class','Time'], axis=1), df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")


Train: (226515, 29), Test: (56629, 29)


Scale features and save scaler

In [2]:

scaler = StandardScaler().fit(X_train)
X_train_scaled, X_test_scaled = scaler.transform(X_train), scaler.transform(X_test); joblib.dump(scaler, "scaler.joblib")


NameError: name 'StandardScaler' is not defined

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_res, y_train_res = SMOTE().fit_resample(X_train, y_train)


Random Forest model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report

# Load and split
df = pd.read_csv("creditcard.csv")
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Class','Time'], axis=1), df['Class'], test_size=0.2, random_state=42, stratify=df['Class'])

# Scale
scaler = StandardScaler().fit(X_train)
X_train_scaled, X_test_scaled = scaler.transform(X_train), scaler.transform(X_test)

# SMOTE + RandomForest (fast)
X_res, y_res = SMOTE(random_state=42).fit_resample(X_train_scaled, y_train)
rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1).fit(X_res, y_res)  # fewer trees + parallel

# Predict + report
y_pred = rf.predict(X_test_scaled)
print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

           0     0.9997    0.9998    0.9997     56864
           1     0.8696    0.8163    0.8421        98

    accuracy                         0.9995     56962
   macro avg     0.9346    0.9081    0.9209     56962
weighted avg     0.9995    0.9995    0.9995     56962



Predict + Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, rf.predict(X_test)))


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


[[56848    16]
 [   27    71]]


XGBoost with imbalance handling

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(scale_pos_weight=10).fit(X_train, y_train)


Precision + Recall

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(y_test, rf.predict(X_test)))


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


(array([0.99952527, 0.81609195]), array([0.99971863, 0.7244898 ]), array([0.99962194, 0.76756757]), array([56864,    98]))


KMeans anomaly detection

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=2).fit(X_train)
pred = (km.predict(X_test) != km.predict(X_test[y_test==0])[0]).astype(int)


Save model

In [ ]:
import joblib; joblib.dump(rf, "fraud_model.joblib")


['fraud_model.joblib']